In [1]:
import os
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd

## Carrega dados

In [2]:
diretorio_covid = os.fsencode('dados/covid')
df_covid_base = None
for arquivo in os.listdir(diretorio_covid):
    nome_arquivo = 'dados/covid/' + os.fsdecode(arquivo)
    df_temp = pd.read_csv(nome_arquivo, decimal=',', sep=';')
    if df_covid_base is None:
        df_covid_base = df_temp
    else:
        df_covid_base = pd.concat([df_covid_base, df_temp])

df_covid_base.data = pd.to_datetime(df_covid_base.data, infer_datetime_format=True)
df_covid_base = df_covid_base.sort_values('data')

In [37]:
df_covid_base

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0,0,0,0.0,0.0,NaN
2615,Nordeste,BA,NaN,29,NaN,NaN,NaN,2020-02-25,9,14873064.0,0.0,0,0,0,NaN,NaN,NaN
2457,Nordeste,SE,NaN,28,NaN,NaN,NaN,2020-02-25,9,2298696.0,0.0,0,0,0,NaN,NaN,NaN
2299,Nordeste,AL,NaN,27,NaN,NaN,NaN,2020-02-25,9,3337357.0,0.0,0,0,0,NaN,NaN,NaN
2141,Nordeste,PE,NaN,26,NaN,NaN,NaN,2020-02-25,9,9557071.0,0.0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352641,Sudeste,MG,São Sebastião da Bela Vista,31,316440.0,31007.0,POUSO ALEGRE,2022-04-26,17,5504.0,905.0,0,16,0,NaN,NaN,0.0
441613,Sudeste,SP,Riolândia,35,354420.0,35157.0,VOTUPORANGA,2022-04-26,17,12518.0,2468.0,0,32,0,NaN,NaN,0.0
648557,Centro-Oeste,GO,São Luiz do Norte,52,522015.0,52013.0,SAO PATRICIO I,2022-04-26,17,5167.0,942.0,0,20,0,NaN,NaN,0.0
515853,Sul,SC,Forquilhinha,42,420545.0,42015.0,CARBONIFERA,2022-04-26,17,26793.0,8197.0,1,64,0,NaN,NaN,0.0


In [38]:
df_municipios = df_covid_base[df_covid_base.municipio.isnull() == False].copy()
df_brasil = df_municipios.groupby(['data', 'semanaEpi']).sum()[['populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].reset_index()
df_regioes = df_municipios.groupby(['regiao', 'data', 'semanaEpi']).sum()[['populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].reset_index()

In [39]:
df_regioes

,regiao,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
0,Centro-Oeste,2020-03-27,13,16297074.0,0.0,0,0,0
1,Centro-Oeste,2020-03-28,13,16297074.0,360.0,360,1,1
2,Centro-Oeste,2020-03-29,14,16297074.0,399.0,39,2,1
3,Centro-Oeste,2020-03-30,14,16297074.0,435.0,36,2,0
4,Centro-Oeste,2020-03-31,14,16297074.0,470.0,35,5,3
...,...,...,...,...,...,...,...,...
3800,Sul,2022-04-22,16,29975984.0,6446203.0,1669,103827,4
3801,Sul,2022-04-23,16,29975984.0,6450865.0,4662,103837,10
3802,Sul,2022-04-24,17,29975984.0,6452486.0,1621,103840,3
3803,Sul,2022-04-25,17,29975984.0,6454099.0,1613,103856,16


## Cria gráficos

In [40]:
figura_obitos_brasil = px.line(df_brasil, x='data', y='obitosNovos')
figura_obitos_brasil.show()

In [45]:
df_regioes['taxaObitosAcumulado'] = df_regioes.obitosAcumulado / df_regioes.populacaoTCU2019 * 100000
figura_obitos_regioes = px.line(df_regioes, x='data', y='taxaObitosAcumulado', color='regiao')
figura_obitos_regioes.show()

## Exibir painel Covid-BR

In [47]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Painel Covid-BR"),
    html.Div(children=[
        dcc.Graph(id='figura_obitos_brasil', figure=figura_obitos_brasil, style={'display': 'inline-block'}),
        dcc.Graph(id='figura_obitos_regioes', figure=figura_obitos_regioes, style={'display': 'inline-block'})
    ])
])
app.run_server(mode='inline')